# LFT AI - Lightweight Fog Testbed com IA

Este notebook permite usar o LFT com geração automática de topologias usando IA.

## Configuração Inicial

In [ ]:
# Verificar se GPU está disponível
import torch
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Instalar dependências
!pip install transformers torch accelerate bitsandbytes huggingface_hub

In [ ]:
# Clonar o repositório LFT
!git clone https://github.com/dcasseb/lft_ai.git
%cd lft_ai

In [ ]:
# Instalar o LFT
!pip install -e .

## Configurar Token do Hugging Face

Você precisa de um token do Hugging Face com acesso aos modelos. Obtenha em: https://huggingface.co/settings/tokens

In [ ]:
# Configurar token do Hugging Face
import os

# Substitua "seu_token_aqui" pelo seu token real
HF_TOKEN = "seu_token_aqui"
os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN
os.environ["HF_TOKEN"] = HF_TOKEN  # Backup para compatibilidade

print("Token configurado!")
print(f"Token verificado: {os.getenv("HUGGING_FACE_HUB_TOKEN") is not None}")

## Verificar Configuração do Token

In [ ]:
# Verificar se o token está configurado corretamente
import os

hf_token = os.getenv("HUGGING_FACE_HUB_TOKEN")
hf_token_backup = os.getenv("HF_TOKEN")

print(f"HUGGING_FACE_HUB_TOKEN configurado: {hf_token is not None}")
print(f"HF_TOKEN configurado: {hf_token_backup is not None}")

if hf_token and hf_token != "seu_token_aqui":
    print("Token configurado corretamente!")
else:
    print("Token não configurado. Configure o token na célula anterior.")
    print("Obtenha seu token em: https://huggingface.co/settings/tokens")

## Testar o Gerador de IA

In [ ]:
# Importar o gerador de IA
from profissa_lft.ai_generator import AITopologyGenerator

# Criar instância do gerador
try:
    # Tentar usar API primeiro
    generator = AITopologyGenerator()
    print("Gerador de IA inicializado com sucesso usando Hugging Face API!")
except Exception as e:
    print(f"Erro ao inicializar com API: {e}")
    print("
Tentando usar modelo local...")
    try:
        # Fallback para modelo local
        generator = AITopologyGenerator(use_hf_api=False)
        print("Gerador de IA inicializado com sucesso usando modelo local!")
    except Exception as e2:
        print(f"Erro ao inicializar modelo local: {e2}")
        print("
Verifique se o token está configurado corretamente ou se há GPU disponível.")

In [ ]:
# Testar geração de código
prompt = "Crie uma topologia simples com 2 hosts conectados por um switch"

try:
    code = generator.generate_topology(prompt)
    print("Código gerado:")
    print("=" * 50)
    print(code)
    print("=" * 50)
except Exception as e:
    print(f"Erro: {e}")

## Usar o CLI Interativo

In [ ]:
# Executar CLI interativo
!python -m profissa_lft.cli interactive

## Gerar Topologia Específica

In [ ]:
# Gerar topologia de rede 4G
prompt_4g = """
Crie uma topologia de rede 4G com:
- 1 eNodeB (estação base)
- 1 EPC (Evolved Packet Core)
- 2 UEs (User Equipment)
- 1 servidor web
Todos conectados através de switches OpenFlow.
"""

try:
    code_4g = generator.generate_topology(prompt_4g)
    
    # Salvar código gerado
    with open("topologia_4g_gerada.py", "w") as f:
        f.write(code_4g)
    
    print("Topologia 4G gerada e salva em "topologia_4g_gerada.py"")
    print("
Código:")
    print(code_4g)
    
except Exception as e:
    print(f"Erro: {e}")

## Salvar no Google Drive (Opcional)

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount("/content/drive")

# Copiar arquivos para o Drive
!cp -r /content/lft_ai /content/drive/MyDrive/
print("Arquivos copiados para o Google Drive!")

## Exemplo de Uso Completo

In [ ]:
# Exemplo completo: gerar e executar topologia
prompt_exemplo = """
Crie uma topologia de teste com:
- 3 hosts (h1, h2, h3)
- 1 switch OpenFlow
- h1 e h2 conectados ao switch
- h3 conectado diretamente a h1
- Configurar IPs: 10.0.0.1/24, 10.0.0.2/24, 10.0.0.3/24
"""

try:
    # Gerar código
    codigo = generator.generate_topology(prompt_exemplo)
    
    # Salvar arquivo
    with open("exemplo_topologia.py", "w") as f:
        f.write(codigo)
    
    print("Topologia gerada com sucesso!")
    print("
Para executar:")
    print("python exemplo_topologia.py")
    
except Exception as e:
    print(f"Erro na geração: {e}")